In [1]:
from glob import glob
from os import path
import numpy as np
from halotools.sim_manager import CachedHaloCatalog, RockstarHlistReader, UserSuppliedPtclCatalog
from helpers.readGadgetSnapshot import readGadgetSnapshot
from AbundanceMatching import *

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


Similar to the other catalog notebook, I'll be loading a catalog of halos then abundance matching them with a luminosity function. However, this time I'm loading a catalog that we have on disk at SLAC instead of one in the cloud. This will require one of Yao's other packages, [helpers](https://bitbucket.org/yymao/helpers/src). After it's loaded, I'll use [Yao's code](https://bitbucket.org/yymao/abundancematching) to do the abundance matching.

In [2]:
simname = 'chinchilla'
Lbox = 400
npart = 2048

In [3]:
z = 0.2
a = 1/(1+z)

In [4]:
HLIST_COLS = {'halo_id': (1, 'i8'), 'halo_upid': (6, 'i8'),
              'halo_x': (17, 'f4'), 'halo_y': (18, 'f4'), 'halo_z': (19, 'f4'),
              'halo_vx': (20, 'f4'), 'halo_vy': (21, 'f4'), 'halo_vz': (22, 'f4'),
              'halo_mvir': (10, 'f4'), 'halo_rvir': (11, 'f4'), 'halo_rs':(12, 'f4'),
              'halo_snapnum': (31, 'i8'),'halo_vpeak':(57, 'f8'), 'halo_halfmass_scale': (58, 'f4')}

In [5]:
halo_finder = 'rockstar'

#may also work for 1050 too.
pmass = 1.44390e+08*((Lbox / 125.0) ** 3) * ((1024.0/npart)** 3)

In [ ]:
path_to_cache_local = '' #UPDATE

In [6]:
tmp_fnames = sorted(glob(loc + 'hlist_*.list'))  # snag all the hlists
available_fnames = [fname[len(loc):] for fname in tmp_fnames]  # just want the names in the dir
available_scale_factors = [float(fname[6:-5]) for fname in available_fnames]  # pull out scale factors
 
idx = np.argmin(np.abs(np.array(available_scale_factors) - a))

In [ ]:
path_to_particles = '' #update
all_particles = np.loadtxt(path_to_particles)

In [ ]:
ptcl_catalog = UserSuppliedPtclCatalog(redshift=z, Lbox=Lbox, particle_mass=pmass,\
                                       x=all_particles[:,0], y=all_particles[:,1], z=all_particles[:,2])
ptcl_cache_loc ='' #update
ptcl_cache_filename = ptcl_cache_loc % (a, simname, version_name)  # make sure we don't have redunancies.
ptcl_catalog.add_ptclcat_to_cache(ptcl_cache_filename, simname, version_name, 'p=%f'%p)

Now we've cached the halo catalog and the particles. You'll probably want to turn off the above cells (the two directly above this one and the one four above) as they only have to be run once. Now let's do the matching. 

In [ ]:
#run me the first time only
halocat = CachedHaloCatalog(path_to_cache_local, update_cached_fname=True)

In [ ]:
halocat.halo_table.colnames

Loading up a luminosity function. This one is from GAMA Survey, Loveday 2015 (got it from Joe). It is tweaked from the one from the other notebook to reflect the new redshfit range.

In [ ]:
lf = np.genfromtxt('lf_r_sersic_z_0.2_0.3.dat', skip_header=True)[:,1:3]
#lf = np.genfromtxt('lf_r_sersic_z_0.1_0.2.dat', skip_header=True)[:,1:3]

In [ ]:
lf

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
use_up_to = -6
af = AbundanceFunction(lf[:use_up_to,0], lf[:use_up_to,1], (-27, -18),faint_end_fit_points = 7 )

In [ ]:
plt.semilogy(lf[:,0], lf[:,1])
x = np.linspace(-27, -18, 101)
plt.semilogy(x, af(x))

In [ ]:
print max(af(x))

In [ ]:
scatter = 0.2
remainder = af.deconvolute(scatter*LF_SCATTER_MULT, 20)

In [ ]:
x, nd = af.get_number_density_table()
plt.plot(x, remainder/nd);

In [ ]:
max(nd)

In [ ]:
halos = np.array(halocat.halo_table)
    
nd_halos = calc_number_densities(halos['halo_vpeak'], halocat.Lbox[0])

In [ ]:
full_catalog = af.match(nd_halos, scatter*LF_SCATTER_MULT)

In [ ]:
mag_cut = -18 
valid_idxs = np.logical_and(~np.isnan(full_catalog), full_catalog <= mag_cut)
catalog = full_catalog[valid_idxs]

In [ ]:
plt.hist(-1*catalog, bins = 30);

We want this plot to approximately match this one from Song. 

In [ ]:
song_cat = np.genfromtxt('gama_z0.1_0.3_rband_absmag.ascii')

In [ ]:
mags = song_cat[song_cat[:,-1] > -10000,-1 ]

In [ ]:
#plt.hist(-1*catalog, bins = 30)
plt.hist(-1*mags, bins = 30);
plt.xlim([18,25]);

In [ ]:
#downsample here
hist, bin_edges = np.histogram(-1*mags, bins = 30)
downsample_idxs = set()
makeshift_hist = []
for idx, count in enumerate(hist):
    low, high = bin_edges[idx:idx+2]
    in_bin_idxs = np.where(np.logical_and(low <= -1*catalog, -1*catalog <= high))[0]
    if in_bin_idxs.shape[0] <= count:
        sampled_idxs = in_bin_idxs
    else:
        sampled_idxs = np.random.choice(in_bin_idxs, size = count, replace = False)

    downsample_idxs = downsample_idxs | set(sampled_idxs)
    
downsampled_catalog = catalog[np.array(list(downsample_idxs), dtype=int)]

In [ ]:
plt.hist(-1*downsampled_catalog, bins = bin_edges)
plt.hist(-1*mags, bins = bin_edges, alpha = 0.5);
plt.xlim([18,25]);

Yeah! let's save the indexs. 

In [ ]:
downsampled_idxs_array = np.zeros_like(valid_idxs, dtype=bool)
counter = 0
for idx, val in enumerate(valid_idxs):
    if val:
        if counter in downsample_idxs:
            downsampled_idxs_array[idx] = True
        counter+=1


In [ ]:
downsampled_idxs_array= np.where(valid_idxs)[0][np.array(list(downsample_idxs))]

In [ ]:
plt.hist(-1*full_catalog[downsampled_idxs_array], bins = 30);

In [ ]:
np.savetxt('gama_matched_catalog.npy', full_catalog)
np.savetxt('gama_matched_catalog_idxs.npy', downsampled_idxs_array)